In [10]:
import netCDF4 as nc
import numpy as np
from netCDF4 import Dataset

from os import listdir
import dateutil.parser  #引入这个库来将日期字符串转成统一的datatime时间格式

import json
cityList = ['三亚', '海口', '湛江', '茂名', '江门', '珠海', '南宁', '深圳', '云浮', '惠州', 
            '广州', '梧州', '佛山', '东莞', '肇庆', '揭阳', '汕头', '河源', '玉溪', '梅州', 
            '潮州', '清远', '河池', '漳州', '贺州', '柳州', '韶关', '桂林', '泉州', '昆明', 
            '曲靖', '龙岩', '厦门', '赣州', '大理', '郴州', '福州', '六盘水', '三明', '怀化', 
            '丽江', '邵阳', '吉安', '攀枝花', '凉山', '衡阳', '株洲', '贵阳', '宁德', '抚州', 
            '萍乡', '温州', '遵义', '铜仁', '湘潭', '新余', '林芝', '宜春', '泸州', '湘西土家族苗族自治州', 
            '上饶', '长沙', '鹰潭', '益阳', '台州', '重庆', '南昌', '衢州', '乐山', '常德', 
            '岳阳', '金华', '九江', '景德镇', '宁波', '自贡', '张家界', '咸宁', '拉萨', '内江', 
            '杭州', '绍兴', '眉山', '黄山', '荆州', '黄石', '池州', '舟山', '黄冈', '安庆', 
            '宜昌', '武汉', '鄂州', '宣城', '成都', '潜江', '遂宁', '达州', '嘉兴', '荆门', 
            '孝感', '湖州', '德阳', '绵阳', '芜湖', '上海', '铜陵', '苏州', '六安', '合肥', 
            '襄阳', '南京', '常州', '无锡', '鞍山', '随州', '信阳', '十堰', '镇江', '南通', 
            '滁州', '泰州', '扬州', '南阳', '驻马店', '阜阳', '淮南', '陇南', '盐城', '海西', 
            '蚌埠', '淮安', '亳州', '平顶山', '周口', '宿迁', '淮北', '漯河', '宝鸡', '三门峡', 
            '洛阳', '西安', '许昌', '商丘', '徐州', '连云港', '天水', '咸阳', '渭南', '开封', 
            '郑州', '临沂', '济宁', '枣庄', '运城', '菏泽', '焦作', '济源', '新乡', '日照', 
            '晋城', '安阳', '临汾', '濮阳', '喀什', '白银', '鹤壁', '青岛', '兰州', '泰安', 
            '潍坊', '长治', '聊城', '淄博', '邯郸', '济南', '中卫', '西宁', '德州', '烟台', 
            '晋中', '威海', '武威', '邢台', '滨州', '衡水', '石家庄', '太原', '沧州', '银川', 
            '阳泉', '酒泉', '忻州', '保定', '大连', '廊坊', '天津', '石嘴山', '唐山', '乌海', 
            '大同', '北京', '秦皇岛', '阿克苏', '嘉峪关', '呼和浩特', '丹东', '葫芦岛', '营口', 
            '承德', '包头', '阿拉尔', '盘锦', '辽阳', '锦州', '本溪', '通化', '吉林', '沈阳', 
            '赤峰', '抚顺', '白山', '阜新', '铁岭', '延边', '辽源', '乌鲁木齐', '昌吉', '长春', 
            '牡丹江', '塔城', '克拉玛依', '五家渠', '哈尔滨', '石河子', '大庆', '七台河', '佳木斯', 
            '齐齐哈尔','巴音郭楞蒙古自治州']
# 记录城市对应的lon和lat维的索引i j                
ijDict = {}
for city in cityList:
    ijDict[city] = []
with open("/Users/rumeng/Downloads/qingzang/test/ij2city2.txt", "r") as f:
    for line in f.readlines():
        line = line.strip('\n').split()
        if line[2] in cityList:
            ijDict[line[2]].append([int(line[0]),int(line[1])])
        

In [2]:
# 获得每月的天数
def getDays(year,month):
#     year=int(dateStr[:4])
#     month=int(dateStr[4:])
    if month in [1,3,5,7,8,10,12]:
        return 31
    elif month in [4,6,9,11]:
        return 30
    else:
        if year%4!=0:
            return 28
        elif year%100==0 and year%400!=0:
            return 28
        else:
            return 29

In [3]:
'''
path='/Users/rumeng/Downloads/qingzang/forcing_03hr/2'
labelpath='/Users/rumeng/Downloads/qingzang/forcing_03hr/2/'

# filelist = listdir(path)

data=[]
cityDate={}
objPath='/Users/rumeng/Downloads/qingzang/trainSet2/weather/'
yearInfo=['2011','2012','2013','2014','2015','2016','2017','2018']
monthInfo=['01','02','03','04','05','06','07','08','09','10','11','12']
fileName='temp_CMFD_V0106_B-01_03hr_010deg_'
# fileName='temp_ITPCAS-CMFD_V0106_B-01_03hr_010deg_'
for y in yearInfo:
    for m in monthInfo:
        year=int(y)
        month=int(m)
        days=getDays(year,month)
        print(year,month,days)

        data=[]
        cityDate={}

        ncTemp=Dataset(labelpath+fileName+y+m+'.nc')
        time = nc.num2date(ncTemp.variables['time'][:],'hours since 1900-1-1 00:00:00').data

        all_times = ncTemp.variables['time']
        sdt = dateutil.parser.parse(str(year)+'-'+str(month)+"-1T00:00:00")#"2000-1-1T00:00:00"
        edt = dateutil.parser.parse(str(year)+'-'+str(month)+'-'+str(days)+"T21:00:00")#"2000-1-31T21:00:00"
        st_idx = nc.date2index(sdt, all_times)
        et_idx = nc.date2index(edt, all_times)
        temp = ncTemp.variables['temp'][st_idx:et_idx+1,:,:].data
    
        for city in ijDict.keys():
    #         print(city)
            for i in ijDict[city]:
                for j in range(days*8):
                    if temp[j][i[0]][i[1]]==-32767:
                        continue
                    year=time[j].year
                    month=time[j].month
                    day=time[j].day  
                    thisDate=str(year)+'-'+str(month)+'-'+str(day)
                    thisTemp=temp[j][i[0]][i[1]]
                    if city+thisDate not in cityDate.keys():
                        cityDate[city+thisDate]=len(data)
                        data.append({"city":city,"date":thisDate,"temp":{"minTemp":thisTemp,"maxTemp":thisTemp}})
                    else:
                        if thisTemp>data[cityDate[city+thisDate]]["temp"]["maxTemp"]:
                            data[cityDate[city+thisDate]]["temp"]["maxTemp"]=thisTemp
                        if thisTemp<data[cityDate[city+thisDate]]["temp"]["minTemp"]:
                            data[cityDate[city+thisDate]]["temp"]["minTemp"]=thisTemp

        length=len(data)
        for i in range(length):
            data[i]["temp"]["maxTemp"]=str(data[i]["temp"]["maxTemp"])
            data[i]["temp"]["minTemp"]=str(data[i]["temp"]["minTemp"])
            city=data[i]["city"]
            j = json.dumps(data[i],ensure_ascii=False)        
          # print(j)  
            with open(objPath+city+'Temp.json', 'a',encoding='utf-8-sig') as json_file:
                json_file.write(j)
                json_file.write('\n') 
'''

2011 1 31


KeyboardInterrupt: 

In [7]:
# 处理两日间温差 第二天的平均温度-第一天的平均温度
'''
sourcePath = '/Users/rumeng/Downloads/qingzang/trainSet2/weather/1/'
objPath = '/Users/rumeng/Downloads/qingzang/trainSet2/weather/2/'
filelist = listdir(sourcePath)
for city in ijDict.keys():
    print(city)
    if city+'Temp.json' not in filelist:
        continue
    with open(sourcePath+city+'Temp.json', 'r',encoding='utf-8-sig') as f:
        lines = f.readlines()
        wencha = 0
        pastAvgTemp =0
        for line in lines:
            climate=json.loads(line)
            avgTemp = (float(climate['temp']['maxTemp'])+float(climate['temp']['minTemp']))/2
            if pastAvgTemp==0:
                wencha=0
            else:
                wencha = avgTemp-pastAvgTemp
            pastAvgTemp = avgTemp
            climate['tempDiff'] = str(round(wencha,2))
            j = json.dumps(climate,ensure_ascii=False)     
            with open(objPath+city+'.json', 'a',encoding='utf-8-sig') as json_file:
                json_file.write(j)
                json_file.write('\n') 
'''

三亚
海口
湛江
茂名
江门
珠海
南宁
深圳
云浮
惠州
广州
梧州
佛山
东莞
肇庆
揭阳
汕头
河源
玉溪
梅州
潮州
清远
河池
漳州
贺州
柳州
韶关
桂林
泉州
昆明
曲靖
龙岩
厦门
赣州
大理
郴州
福州
六盘水
三明
怀化
丽江
邵阳
吉安
攀枝花
凉山
衡阳
株洲
贵阳
宁德
抚州
萍乡
温州
遵义
铜仁
湘潭
新余
林芝
宜春
泸州
湘西土家族苗族自治州
上饶
长沙
鹰潭
益阳
台州
重庆
南昌
衢州
乐山
常德
岳阳
金华
九江
景德镇
宁波
自贡
张家界
咸宁
拉萨
内江
杭州
绍兴
眉山
黄山
荆州
黄石
池州
舟山
黄冈
安庆
宜昌
武汉
鄂州
宣城
成都
潜江
遂宁
达州
嘉兴
荆门
孝感
湖州
德阳
绵阳
芜湖
上海
铜陵
苏州
六安
合肥
襄阳
南京
常州
无锡
鞍山
随州
信阳
十堰
镇江
南通
滁州
泰州
扬州
南阳
驻马店
阜阳
淮南
陇南
盐城
海西
蚌埠
淮安
亳州
平顶山
周口
宿迁
淮北
漯河
宝鸡
三门峡
洛阳
西安
许昌
商丘
徐州
连云港
天水
咸阳
渭南
开封
郑州
临沂
济宁
枣庄
运城
菏泽
焦作
济源
新乡
日照
晋城
安阳
临汾
濮阳
喀什
白银
鹤壁
青岛
兰州
泰安
潍坊
长治
聊城
淄博
邯郸
济南
中卫
西宁
德州
烟台
晋中
威海
武威
邢台
滨州
衡水
石家庄
太原
沧州
银川
阳泉
酒泉
忻州
保定
大连
廊坊
天津
石嘴山
唐山
乌海
大同
北京
秦皇岛
阿克苏
嘉峪关
呼和浩特
丹东
葫芦岛
营口
承德
包头
阿拉尔
盘锦
辽阳
锦州
本溪
通化
吉林
沈阳
赤峰
抚顺
白山
阜新
铁岭
延边
辽源
乌鲁木齐
昌吉
长春
牡丹江
塔城
克拉玛依
五家渠
哈尔滨
石河子
大庆
七台河
佳木斯
齐齐哈尔
巴音郭楞蒙古自治州


In [4]:
import json
'''
#/Users/rumeng/Downloads/qingzang/forcing_01dy/lrad
path='/Users/rumeng/Downloads/qingzang/forcing_01dy'
labelpath='/Users/rumeng/Downloads/qingzang/forcing_01dy/'

# params=['wind']
##wind风速
##lrad地面向下长波辐射
##prec降水率 Precipitation rate
##pres近地表气压 Near surface air pressure
##shum近地表空气比湿度 Near surface air specific humidity
##srad地面向下短波辐射 Surface downward shortwave radiation
params=['wind','lrad','prec','pres','shum','srad']

data=[]
cityDate={}
objPath='/Users/rumeng/Downloads/qingzang/trainSet2/weather/'
filelist1 = listdir(objPath+'2/')
filelist2 = listdir(objPath+'3/')
yearInfo=['2011','2012','2013','2014','2015','2016','2017','2018']
monthInfo=['01','02','03','04','05','06','07','08','09','10','11','12']
fileName='_CMFD_V0106_B-01_01dy_010deg_'
# /Users/rumeng/Downloads/qingzang/forcing_01dy/wind/wind_CMFD_V0106_B-01_01dy_010deg_200001-200012.nc
notHave = []
for city in ijDict.keys():
    if city+'.json' in filelist2:
        continue
    if city+'.json' not in filelist1:
        notHave.append(city)
        continue
    list1=[] # 存储信息
    with open(objPath+'2/'+city+'.json', 'r',encoding='utf-8-sig') as f:
        lines = f.readlines()
        for line in lines:
            climateInfo=json.loads(line)
            list1.append(climateInfo)
    passdays=0
    for y in yearInfo:
        numOfGuance=len(ijDict[city])
        sumOfGuance=0
        
        thisday=passdays
        
        for paramName in params:
            print(city,y,paramName)
            ncTemp=Dataset(labelpath+paramName+'/'+paramName+fileName+y+'01-'+y+'12.nc')
            days=len(ncTemp.variables['time'])
            all_times = ncTemp.variables['time']
            sdt = dateutil.parser.parse(y+'-1-1T10:30:00')#"2000-1-1T00:00:00"
            edt = dateutil.parser.parse(y+'-12-31T10:30:00')#"2000-1-31T21:00:00"
            st_idx = nc.date2index(sdt, all_times)
            et_idx = nc.date2index(edt, all_times)
            time = nc.num2date(ncTemp.variables['time'][:],'hours since 1900-1-1 00:00:00').data
            
            paramInfo = ncTemp.variables[paramName][st_idx:et_idx+1,:,:].data
            thisday=passdays
            for j in range(days):
                numOfGuance=len(ijDict[city])
                sumOfGuance=0
                
                year=time[j].year
                month=time[j].month
                day=time[j].day  
                
                for i in ijDict[city]: 
                    if paramInfo[j][i[0]][i[1]]==-32767:
                        numOfGuance-=1
                        continue
                    else:
                        if paramInfo[j][i[0]][i[1]]<0:
                            print('paramInfo',paramInfo[j][i[0]][i[1]])
                        sumOfGuance+=paramInfo[j][i[0]][i[1]]
                if str(year)+'-'+str(month)+'-'+str(day)!=list1[thisday]["date"]:
                    print('日期不匹配')
                if numOfGuance!=0:
                    avgParam=round(sumOfGuance/numOfGuance,2)
#                     if avgParam>8 or avgParam<=0:
#                         print('avgParam:',avgParam)
                    list1[thisday][paramName] = str(avgParam)
#                     print(sumOfGuance,numOfGuance)
                else:
                    print('numOfGuance为0',city,y,j)
                    list1[thisday][paramName] = '0.00'
                thisday+=1
        passdays+=days
        
    length=len(list1)
    for k in range(length):
        j = json.dumps(list1[k],ensure_ascii=False)  
        with open(objPath+'3/'+city+'.json', 'a',encoding='utf-8-sig') as json_file:
            json_file.write(j)
            json_file.write('\n')
'''

黄冈 2011 wind
黄冈 2011 lrad
黄冈 2011 prec
黄冈 2011 pres
黄冈 2011 shum
黄冈 2011 srad
黄冈 2012 wind
黄冈 2012 lrad
黄冈 2012 prec
黄冈 2012 pres
黄冈 2012 shum
黄冈 2012 srad
黄冈 2013 wind
黄冈 2013 lrad
黄冈 2013 prec
黄冈 2013 pres
黄冈 2013 shum
黄冈 2013 srad
黄冈 2014 wind
黄冈 2014 lrad
黄冈 2014 prec
黄冈 2014 pres
黄冈 2014 shum
黄冈 2014 srad
黄冈 2015 wind
黄冈 2015 lrad
黄冈 2015 prec
黄冈 2015 pres
黄冈 2015 shum
黄冈 2015 srad
黄冈 2016 wind
黄冈 2016 lrad
黄冈 2016 prec
黄冈 2016 pres
黄冈 2016 shum
黄冈 2016 srad
黄冈 2017 wind
黄冈 2017 lrad
黄冈 2017 prec
黄冈 2017 pres
黄冈 2017 shum
黄冈 2017 srad
黄冈 2018 wind
黄冈 2018 lrad
黄冈 2018 prec
黄冈 2018 pres
黄冈 2018 shum
黄冈 2018 srad
安庆 2011 wind
安庆 2011 lrad
安庆 2011 prec
安庆 2011 pres
安庆 2011 shum
安庆 2011 srad
安庆 2012 wind
安庆 2012 lrad
安庆 2012 prec
安庆 2012 pres
安庆 2012 shum
安庆 2012 srad
安庆 2013 wind
安庆 2013 lrad
安庆 2013 prec
安庆 2013 pres
安庆 2013 shum
安庆 2013 srad
安庆 2014 wind
安庆 2014 lrad
安庆 2014 prec
安庆 2014 pres
安庆 2014 shum
安庆 2014 srad
安庆 2015 wind
安庆 2015 lrad
安庆 2015 prec
安庆 2015 pres
安庆 2015 shum

湖州 2012 lrad
湖州 2012 prec
湖州 2012 pres
湖州 2012 shum
湖州 2012 srad
湖州 2013 wind
湖州 2013 lrad
湖州 2013 prec
湖州 2013 pres
湖州 2013 shum
湖州 2013 srad
湖州 2014 wind
湖州 2014 lrad
湖州 2014 prec
湖州 2014 pres
湖州 2014 shum
湖州 2014 srad
湖州 2015 wind
湖州 2015 lrad
湖州 2015 prec
湖州 2015 pres
湖州 2015 shum
湖州 2015 srad
湖州 2016 wind
湖州 2016 lrad
湖州 2016 prec
湖州 2016 pres
湖州 2016 shum
湖州 2016 srad
湖州 2017 wind
湖州 2017 lrad
湖州 2017 prec
湖州 2017 pres
湖州 2017 shum
湖州 2017 srad
湖州 2018 wind
湖州 2018 lrad
湖州 2018 prec
湖州 2018 pres
湖州 2018 shum
湖州 2018 srad
德阳 2011 wind
德阳 2011 lrad
德阳 2011 prec
德阳 2011 pres
德阳 2011 shum
德阳 2011 srad
德阳 2012 wind
德阳 2012 lrad
德阳 2012 prec
德阳 2012 pres
德阳 2012 shum
德阳 2012 srad
德阳 2013 wind
德阳 2013 lrad
德阳 2013 prec
德阳 2013 pres
德阳 2013 shum
德阳 2013 srad
德阳 2014 wind
德阳 2014 lrad
德阳 2014 prec
德阳 2014 pres
德阳 2014 shum
德阳 2014 srad
德阳 2015 wind
德阳 2015 lrad
德阳 2015 prec
德阳 2015 pres
德阳 2015 shum
德阳 2015 srad
德阳 2016 wind
德阳 2016 lrad
德阳 2016 prec
德阳 2016 pres
德阳 2016 shum
德阳 2016 srad

鞍山 2013 prec
鞍山 2013 pres
鞍山 2013 shum
鞍山 2013 srad
鞍山 2014 wind
鞍山 2014 lrad
鞍山 2014 prec
鞍山 2014 pres
鞍山 2014 shum
鞍山 2014 srad
鞍山 2015 wind
鞍山 2015 lrad
鞍山 2015 prec
鞍山 2015 pres
鞍山 2015 shum
鞍山 2015 srad
鞍山 2016 wind
鞍山 2016 lrad
鞍山 2016 prec
鞍山 2016 pres
鞍山 2016 shum
鞍山 2016 srad
鞍山 2017 wind
鞍山 2017 lrad
鞍山 2017 prec
鞍山 2017 pres
鞍山 2017 shum
鞍山 2017 srad
鞍山 2018 wind
鞍山 2018 lrad
鞍山 2018 prec
鞍山 2018 pres
鞍山 2018 shum
鞍山 2018 srad
随州 2011 wind
随州 2011 lrad
随州 2011 prec
随州 2011 pres
随州 2011 shum
随州 2011 srad
随州 2012 wind
随州 2012 lrad
随州 2012 prec
随州 2012 pres
随州 2012 shum
随州 2012 srad
随州 2013 wind
随州 2013 lrad
随州 2013 prec
随州 2013 pres
随州 2013 shum
随州 2013 srad
随州 2014 wind
随州 2014 lrad
随州 2014 prec
随州 2014 pres
随州 2014 shum
随州 2014 srad
随州 2015 wind
随州 2015 lrad
随州 2015 prec
随州 2015 pres
随州 2015 shum
随州 2015 srad
随州 2016 wind
随州 2016 lrad
随州 2016 prec
随州 2016 pres
随州 2016 shum
随州 2016 srad
随州 2017 wind
随州 2017 lrad
随州 2017 prec
随州 2017 pres
随州 2017 shum
随州 2017 srad
随州 2018 wind

陇南 2013 srad
陇南 2014 wind
陇南 2014 lrad
陇南 2014 prec
陇南 2014 pres
陇南 2014 shum
陇南 2014 srad
陇南 2015 wind
陇南 2015 lrad
陇南 2015 prec
陇南 2015 pres
陇南 2015 shum
陇南 2015 srad
陇南 2016 wind
陇南 2016 lrad
陇南 2016 prec
陇南 2016 pres
陇南 2016 shum
陇南 2016 srad
陇南 2017 wind
陇南 2017 lrad
陇南 2017 prec
陇南 2017 pres
陇南 2017 shum
陇南 2017 srad
陇南 2018 wind
陇南 2018 lrad
陇南 2018 prec
陇南 2018 pres
陇南 2018 shum
陇南 2018 srad
盐城 2011 wind
盐城 2011 lrad
盐城 2011 prec
盐城 2011 pres
盐城 2011 shum
盐城 2011 srad
盐城 2012 wind
盐城 2012 lrad
盐城 2012 prec
盐城 2012 pres
盐城 2012 shum
盐城 2012 srad
盐城 2013 wind
盐城 2013 lrad
盐城 2013 prec
盐城 2013 pres
盐城 2013 shum
盐城 2013 srad
盐城 2014 wind
盐城 2014 lrad
盐城 2014 prec
盐城 2014 pres
盐城 2014 shum
盐城 2014 srad
盐城 2015 wind
盐城 2015 lrad
盐城 2015 prec
盐城 2015 pres
盐城 2015 shum
盐城 2015 srad
盐城 2016 wind
盐城 2016 lrad
盐城 2016 prec
盐城 2016 pres
盐城 2016 shum
盐城 2016 srad
盐城 2017 wind
盐城 2017 lrad
盐城 2017 prec
盐城 2017 pres
盐城 2017 shum
盐城 2017 srad
盐城 2018 wind
盐城 2018 lrad
盐城 2018 prec
盐城 2018 pres

洛阳 2013 shum
洛阳 2013 srad
洛阳 2014 wind
洛阳 2014 lrad
洛阳 2014 prec
洛阳 2014 pres
洛阳 2014 shum
洛阳 2014 srad
洛阳 2015 wind
洛阳 2015 lrad
洛阳 2015 prec
洛阳 2015 pres
洛阳 2015 shum
洛阳 2015 srad
洛阳 2016 wind
洛阳 2016 lrad
洛阳 2016 prec
洛阳 2016 pres
洛阳 2016 shum
洛阳 2016 srad
洛阳 2017 wind
洛阳 2017 lrad
洛阳 2017 prec
洛阳 2017 pres
洛阳 2017 shum
洛阳 2017 srad
洛阳 2018 wind
洛阳 2018 lrad
洛阳 2018 prec
洛阳 2018 pres
洛阳 2018 shum
洛阳 2018 srad
西安 2011 wind
西安 2011 lrad
西安 2011 prec
西安 2011 pres
西安 2011 shum
西安 2011 srad
西安 2012 wind
西安 2012 lrad
西安 2012 prec
西安 2012 pres
西安 2012 shum
西安 2012 srad
西安 2013 wind
西安 2013 lrad
西安 2013 prec
西安 2013 pres
西安 2013 shum
西安 2013 srad
西安 2014 wind
西安 2014 lrad
西安 2014 prec
西安 2014 pres
西安 2014 shum
西安 2014 srad
西安 2015 wind
西安 2015 lrad
西安 2015 prec
西安 2015 pres
西安 2015 shum
西安 2015 srad
西安 2016 wind
西安 2016 lrad
西安 2016 prec
西安 2016 pres
西安 2016 shum
西安 2016 srad
西安 2017 wind
西安 2017 lrad
西安 2017 prec
西安 2017 pres
西安 2017 shum
西安 2017 srad
西安 2018 wind
西安 2018 lrad
西安 2018 prec

...

烟台 2016 pres
烟台 2016 shum
烟台 2016 srad
烟台 2017 wind
烟台 2017 lrad
烟台 2017 prec
烟台 2017 pres
烟台 2017 shum
烟台 2017 srad
烟台 2018 wind
烟台 2018 lrad
烟台 2018 prec
烟台 2018 pres
烟台 2018 shum
烟台 2018 srad
晋中 2011 wind
晋中 2011 lrad
晋中 2011 prec
晋中 2011 pres
晋中 2011 shum
晋中 2011 srad
晋中 2012 wind
晋中 2012 lrad
晋中 2012 prec
晋中 2012 pres
晋中 2012 shum
晋中 2012 srad
晋中 2013 wind
晋中 2013 lrad
晋中 2013 prec
晋中 2013 pres
晋中 2013 shum
晋中 2013 srad
晋中 2014 wind
晋中 2014 lrad
晋中 2014 prec
晋中 2014 pres
晋中 2014 shum
晋中 2014 srad
晋中 2015 wind
晋中 2015 lrad
晋中 2015 prec
晋中 2015 pres
晋中 2015 shum
晋中 2015 srad
晋中 2016 wind
晋中 2016 lrad
晋中 2016 prec
晋中 2016 pres
晋中 2016 shum
晋中 2016 srad
晋中 2017 wind
晋中 2017 lrad
晋中 2017 prec
晋中 2017 pres
晋中 2017 shum
晋中 2017 srad
晋中 2018 wind
晋中 2018 lrad
晋中 2018 prec
晋中 2018 pres
晋中 2018 shum
晋中 2018 srad
威海 2011 wind
威海 2011 lrad
威海 2011 prec
威海 2011 pres
威海 2011 shum
威海 2011 srad
威海 2012 wind
威海 2012 lrad
威海 2012 prec
威海 2012 pres
威海 2012 shum
威海 2012 srad
威海 2013 wind
威海 2013 lrad

忻州 2017 wind
忻州 2017 lrad
忻州 2017 prec
忻州 2017 pres
忻州 2017 shum
忻州 2017 srad
忻州 2018 wind
忻州 2018 lrad
忻州 2018 prec
忻州 2018 pres
忻州 2018 shum
忻州 2018 srad
保定 2011 wind
保定 2011 lrad
保定 2011 prec
保定 2011 pres
保定 2011 shum
保定 2011 srad
保定 2012 wind
保定 2012 lrad
保定 2012 prec
保定 2012 pres
保定 2012 shum
保定 2012 srad
保定 2013 wind
保定 2013 lrad
保定 2013 prec
保定 2013 pres
保定 2013 shum
保定 2013 srad
保定 2014 wind
保定 2014 lrad
保定 2014 prec
保定 2014 pres
保定 2014 shum
保定 2014 srad
保定 2015 wind
保定 2015 lrad
保定 2015 prec
保定 2015 pres
保定 2015 shum
保定 2015 srad
保定 2016 wind
保定 2016 lrad
保定 2016 prec
保定 2016 pres
保定 2016 shum
保定 2016 srad
保定 2017 wind
保定 2017 lrad
保定 2017 prec
保定 2017 pres
保定 2017 shum
保定 2017 srad
保定 2018 wind
保定 2018 lrad
保定 2018 prec
保定 2018 pres
保定 2018 shum
保定 2018 srad
大连 2011 wind
大连 2011 lrad
大连 2011 prec
大连 2011 pres
大连 2011 shum
大连 2011 srad
大连 2012 wind
大连 2012 lrad
大连 2012 prec
大连 2012 pres
大连 2012 shum
大连 2012 srad
大连 2013 wind
大连 2013 lrad
大连 2013 prec
大连 2013 pres
大连 2013 shum

呼和浩特 2015 wind
呼和浩特 2015 lrad
呼和浩特 2015 prec
呼和浩特 2015 pres
呼和浩特 2015 shum
呼和浩特 2015 srad
呼和浩特 2016 wind
呼和浩特 2016 lrad
呼和浩特 2016 prec
呼和浩特 2016 pres
呼和浩特 2016 shum
呼和浩特 2016 srad
呼和浩特 2017 wind
呼和浩特 2017 lrad
呼和浩特 2017 prec
呼和浩特 2017 pres
呼和浩特 2017 shum
呼和浩特 2017 srad
呼和浩特 2018 wind
呼和浩特 2018 lrad
呼和浩特 2018 prec
呼和浩特 2018 pres
呼和浩特 2018 shum
呼和浩特 2018 srad
丹东 2011 wind
丹东 2011 lrad
丹东 2011 prec
丹东 2011 pres
丹东 2011 shum
丹东 2011 srad
丹东 2012 wind
丹东 2012 lrad
丹东 2012 prec
丹东 2012 pres
丹东 2012 shum
丹东 2012 srad
丹东 2013 wind
丹东 2013 lrad
丹东 2013 prec
丹东 2013 pres
丹东 2013 shum
丹东 2013 srad
丹东 2014 wind
丹东 2014 lrad
丹东 2014 prec
丹东 2014 pres
丹东 2014 shum
丹东 2014 srad
丹东 2015 wind
丹东 2015 lrad
丹东 2015 prec
丹东 2015 pres
丹东 2015 shum
丹东 2015 srad
丹东 2016 wind
丹东 2016 lrad
丹东 2016 prec
丹东 2016 pres
丹东 2016 shum
丹东 2016 srad
丹东 2017 wind
丹东 2017 lrad
丹东 2017 prec
丹东 2017 pres
丹东 2017 shum
丹东 2017 srad
丹东 2018 wind
丹东 2018 lrad
丹东 2018 prec
丹东 2018 pres
丹东 2018 shum
丹东 2018 srad
葫芦岛 2011 wind
葫芦

沈阳 2014 prec
沈阳 2014 pres
沈阳 2014 shum
沈阳 2014 srad
沈阳 2015 wind
沈阳 2015 lrad
沈阳 2015 prec
沈阳 2015 pres
沈阳 2015 shum
沈阳 2015 srad
沈阳 2016 wind
沈阳 2016 lrad
沈阳 2016 prec
沈阳 2016 pres
沈阳 2016 shum
沈阳 2016 srad
沈阳 2017 wind
沈阳 2017 lrad
沈阳 2017 prec
沈阳 2017 pres
沈阳 2017 shum
沈阳 2017 srad
沈阳 2018 wind
沈阳 2018 lrad
沈阳 2018 prec
沈阳 2018 pres
沈阳 2018 shum
沈阳 2018 srad
赤峰 2011 wind
赤峰 2011 lrad
赤峰 2011 prec
赤峰 2011 pres
赤峰 2011 shum
赤峰 2011 srad
赤峰 2012 wind
赤峰 2012 lrad
赤峰 2012 prec
赤峰 2012 pres
赤峰 2012 shum
赤峰 2012 srad
赤峰 2013 wind
赤峰 2013 lrad
赤峰 2013 prec
赤峰 2013 pres
赤峰 2013 shum
赤峰 2013 srad
赤峰 2014 wind
赤峰 2014 lrad
赤峰 2014 prec
赤峰 2014 pres
赤峰 2014 shum
赤峰 2014 srad
赤峰 2015 wind
赤峰 2015 lrad
赤峰 2015 prec
赤峰 2015 pres
赤峰 2015 shum
赤峰 2015 srad
赤峰 2016 wind
赤峰 2016 lrad
赤峰 2016 prec
赤峰 2016 pres
赤峰 2016 shum
赤峰 2016 srad
赤峰 2017 wind
赤峰 2017 lrad
赤峰 2017 prec
赤峰 2017 pres
赤峰 2017 shum
赤峰 2017 srad
赤峰 2018 wind
赤峰 2018 lrad
赤峰 2018 prec
赤峰 2018 pres
赤峰 2018 shum
赤峰 2018 srad
抚顺 2011 wind

塔城 2018 prec
塔城 2018 pres
塔城 2018 shum
塔城 2018 srad
克拉玛依 2011 wind
克拉玛依 2011 lrad
克拉玛依 2011 prec
克拉玛依 2011 pres
克拉玛依 2011 shum
克拉玛依 2011 srad
克拉玛依 2012 wind
克拉玛依 2012 lrad
克拉玛依 2012 prec
克拉玛依 2012 pres
克拉玛依 2012 shum
克拉玛依 2012 srad
克拉玛依 2013 wind
克拉玛依 2013 lrad
克拉玛依 2013 prec
克拉玛依 2013 pres
克拉玛依 2013 shum
克拉玛依 2013 srad
克拉玛依 2014 wind
克拉玛依 2014 lrad
克拉玛依 2014 prec
克拉玛依 2014 pres
克拉玛依 2014 shum
克拉玛依 2014 srad
克拉玛依 2015 wind
克拉玛依 2015 lrad
克拉玛依 2015 prec
克拉玛依 2015 pres
克拉玛依 2015 shum
克拉玛依 2015 srad
克拉玛依 2016 wind
克拉玛依 2016 lrad
克拉玛依 2016 prec
克拉玛依 2016 pres
克拉玛依 2016 shum
克拉玛依 2016 srad
克拉玛依 2017 wind
克拉玛依 2017 lrad
克拉玛依 2017 prec
克拉玛依 2017 pres
克拉玛依 2017 shum
克拉玛依 2017 srad
克拉玛依 2018 wind
克拉玛依 2018 lrad
克拉玛依 2018 prec
克拉玛依 2018 pres
克拉玛依 2018 shum
克拉玛依 2018 srad
五家渠 2011 wind
五家渠 2011 lrad
五家渠 2011 prec
五家渠 2011 pres
五家渠 2011 shum
五家渠 2011 srad
五家渠 2012 wind
五家渠 2012 lrad
五家渠 2012 prec
五家渠 2012 pres
五家渠 2012 shum
五家渠 2012 srad
五家渠 2013 wind
五家渠 2013 lrad
五家渠 2013 prec
五家渠 2013 pres
五家渠 

In [11]:
# 数据量>=5的城市名单 共101个
above5 = ['镇江', '杭州', '徐州', '潍坊', '西宁', '嘉兴', '保定', '许昌', '柳州', '株洲', 
          '广州', '东莞', '铜陵', '南阳', '桂林', '包头', '龙岩', '重庆', '哈尔滨', '淄博', 
          '洛阳', '荆门', '兰州', '温州', '湖州', '常州', '绵阳', '石家庄', '银川', '南宁', 
          '衢州', '滨州', '鞍山', '德州', '无锡', '南通', '襄阳', '武汉', '长沙', '大连', 
          '梅州', '沈阳', '乐山', '西安', '南昌', '惠州', '连云港', '福州', '北京', '德阳', 
          '济宁', '郑州', '珠海', '乌鲁木齐', '岳阳', '呼和浩特', '宁波', '拉萨', '赣州', '新乡',
          '天津', '江门', '南京', '长春', '吉林', '济南', '汕头', '太原', '海口', '台州',
          '宝鸡', '青岛', '威海', '昆明', '贵阳', '上海', '益阳', '深圳', '揭阳', '滁州', 
          '绍兴', '唐山', '合肥', '厦门', '苏州', '焦作', '金华', '泉州', '泰州', '衡阳', 
          '漳州', '成都', '芜湖', '扬州', '盐城', '佛山', '烟台', '潮州', '宿迁', '肇庆', 
          '宜昌']
# above5 = ['镇江']
# numLt3 = []   #股价<3
# stkFileList = listdir('/Users/rumeng/Downloads/qingzang/trainSet/stock/')
# for city in above5:
#     with open('/Users/rumeng/Downloads/qingzang/trainSet2/company/'+city+'.json', 'r',encoding='utf-8-sig') as f:
#         for line in f.readlines():
#             companyInfo = json.loads(line)
#             stkcd = companyInfo["股票代码"]
#             if stkcd+'.json' not in stkFileList:
# #                print(stkcd)
#                 continue
#             with open('/Users/rumeng/Downloads/qingzang/trainSet/stock/'+stkcd+'.json', 'r',encoding='utf-8-sig') as f2:
#                 lines2 = f2.readlines()
# #                 print(lines2[len(lines2)-1])
#                 stkInfo = json.loads(lines2[len(lines2)-1])
#                 stkPrice = stkInfo["Clsprc"]
# #                 print(city,stkcd,stkPrice,stkPrice>3,stkInfo["Dsmvosd"])
#                 if stkPrice<1:
#                     numLt3.append([city,stkcd,stkPrice])
# print(len(numLt3))
# print(numLt3)

In [2]:
import pandas as pd
pollutionPath='/Users/rumeng/Downloads/pollution/GW_CITYAIRQUALITY.xlsx'
df = pd.read_excel(pollutionPath)
data=df.values




/Users/rumeng/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [8]:
objPath = '/Users/rumeng/Downloads/qingzang/trainSet2/pollution/'
notInList = []
for i in range(2,len(data)):
    city = data[i][3].replace('市','')
    if city not in above5:
        if city not in notInList:
            notInList.append(notInList)
    myDict = {'date':data[i][0], 'API':data[i][4], 'AQI':data[i][5]}
    j = json.dumps(myDict,ensure_ascii=False)  
    with open(objPath+city+'.json', 'a',encoding='utf-8-sig') as f:
        f.write(j)
        f.write('\n')
print(notInList) 
#     if data[i][1]=='P9714':
#         umdDict = {'Trddt':data[i][0],'UMD1':data[i][8],'UMD2':data[i][9],'UMD3':data[i][10]}
#         j = json.dumps(umdDict,ensure_ascii=False)  
#         with open(objPath5+'umd.json', 'a',encoding='utf-8-sig') as json_file:
#             json_file.write(j)
#             json_file.write('\n') 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
filelist = listdir('/Users/rumeng/Downloads/qingzang/trainSet2/pollution/')
for city in above5:
    if city+'.json' not in filelist:
        print(city)